# Feature Extraction using Unsupervised Methods

This notebook augments the X_train and X_test dataframes with additional features from PCA extraction, MDS, and K-Means clustering techniques.

Additionally, it preserves all objects as python 'pickle' files so that results can be replicated in the future.

In [1]:
#123456789012345678901234567890123456789012345678901234567890123456789012345678

In [2]:
import sys
PATH_TO_MODULES = '../..'
sys.path.insert(0, PATH_TO_MODULES)

from helpers import *
from unsupervised_helpers import *

In [3]:


def extract_features(input_df: pd.DataFrame) -> pd.DataFrame:
    """Extract features from a training dataset using PCA.

    Use this function only on the TRAINING set.
    This function extracts features using PCA and assigns cluster labels
    using K-Means.  It also saves an augmented dataframe as a CSV file, and 
    preserves the objects used to transform the data as 'pickle' files.
    """
    dataset = input_df.copy()
    if 'Unnamed: 0' in dataset.columns:
        dataset.drop(columns=['Unnamed: 0'], inplace=True)
    # Process for PCA
    pca_df = process_for_PCA(dataset)
    # We found that the top 50 PCs explain 85% of the variance
    pca = PCA(50)
    X_PCA = pca.fit_transform(pca_df)
    # Pickle the pca object
    with open(path + 'pca_all.pickle', 'wb') as f:
        pickle.dump(pca, f)
    # Make a dataframe of the top 50 PCs
    PCA_cols = ['PCA_all_PC' + str(i + 1) for i in range(X_PCA.shape[1])]
    top50_PC_all_df = pd.DataFrame(X_PCA, columns=PCA_cols)
    # Use K-Means to assign cluster labels
    kmeans = KMeans(n_clusters=7, init='random', n_init=100, copy_x=False)
    cluster_label = kmeans.fit_predict(X_PCA)
    # Pickle the K-Means object
    with open(path + 'kmeans.pickle', 'wb') as f:
        pickle.dump(kmeans, f)
    # Perform PCA using only the KPI subset
    subset_df = get_KPI(dataset)
    subset_pca_df = process_for_PCA(subset_df)
    pca_KPI = PCA(10)
    X_PCA_KPI = pca_KPI.fit_transform(pca_df)
    # pickle the pca_KPI object
    with open(path + 'pca_KPI.pickle', 'wb') as f:
        pickle.dump(pca_KPI, f)
    # Make a dataframe of the top 10 PCs from the KPI subset
    PCA_KPI_cols = ['PCA_KPI_PC' + str(i + 1) for i in range(X_PCA_KPI.shape[1])]
    top10_PC_KPI_df = pd.DataFrame(X_PCA_KPI, columns=PCA_KPI_cols)
    # Create augmented DataFrame
    augmented_df = pd.concat(
        (
            dataset,
            top50_PC_all_df,
            top10_PC_KPI_df,
            pd.Series(cluster_label, name='Cluster')
        ), axis = 1
    )
    return augmented_df

    
def transform_features(input_df: pd.DataFrame) -> pd.DataFrame:
    """Augment a dataframe with extracted features.

    Use this function on the test set.
    This function uses pickled objects that were used during feature extraction
    on the test set to augment the test set with additional features.
    """
    dataset = input_df.copy()
    if 'Unnamed: 0' in dataset.columns:
        dataset.drop(columns=['Unnamed: 0'], inplace=True)
    # Need to finish this....
    return dataset
    
    


## Load data


In [5]:
# Import merged dataset
path = PATH_TO_MODULES + '/datasets/'
input_filename = 'X_train_filled_KPIs_QoQ.csv'
dataset = pd.read_csv(path + input_filename)
print(f'There are {dataset.shape[0]} rows and {dataset.shape[1]} columns in the dataset.')

There are 1905 rows and 284 columns in the dataset.


In [6]:
augmented_df = extract_features(dataset)

Dropping non-numeric columns.
Remaining dtypes: [dtype('float64') dtype('int64')]
There are 1905 rows and 278 columns in the dataset.
Dropping non-numeric columns.
Remaining dtypes: [dtype('float64')]
There are 1905 rows and 47 columns in the dataset.
